In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
#from torch.optim.lr_scheduler import StepLR

In [2]:
df = pd.read_csv('C:/Users/JenMing/Desktop/MBTI/LSTM/mbti_to_LSTM_DF.csv')
df.head()

,type,posts
0,INFJ,"['INFP', 'INFP', 'INFJ', 'ENFP', 'ISTP', 'INTP..."
1,ENTP,"['INTJ', 'INTP', 'ENFP', 'INTJ', 'INTP', 'INTP..."
2,INTP,"['INTJ', 'INFP', 'INFP', 'INTP', 'INTP', 'INTJ..."
3,INTJ,"['INTJ', 'ISFJ', 'INFP', 'INTP', 'INTP', 'INTP..."
4,ENTJ,"['ENTJ', 'INTP', 'ENFP', 'INTP', 'ENTJ', 'INTJ..."


In [3]:
# 編碼轉換
personality_mapping = {'INFJ': 0,
                        'ENTP': 1,
                        'INTP': 2,
                        'INTJ': 3,
                        'ENTJ': 4,
                        'ENFJ': 5,
                        'INFP': 6,
                        'ENFP': 7,
                        'ISFP': 8,
                        'ISTP': 9,
                        'ISFJ': 10,
                        'ISTJ': 11,
                        'ESTP': 12,
                        'ESFP': 13,
                        'ESTJ': 14,
                        'ESFJ': 15 }

In [4]:
# 資料載入和轉換
encoded_data = []

chars_to_remove = "][' "    

for index, row in df.iterrows():
    mbti_counts = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0}
    mbti_per_count = []
    dialogues = row["posts"] #字串
    target_personality = row["type"]
    for char in chars_to_remove:
        dialogues = dialogues.replace(char, "")
    
    dialogues_list = dialogues.split(',')
    
    
    dialogue_ids = [personality_mapping[personality] for personality in dialogues_list]
    
    for personality_id in dialogue_ids:
        mbti_counts[personality_id] += 1
    
    for i in range(len(personality_mapping)):
        mbti_per_count.append(round(mbti_counts[i]/len(dialogue_ids), 2))
    
    target_personality_id = personality_mapping[target_personality]
    
    encoded_data.append((mbti_per_count, target_personality_id))

In [5]:
encoded_data[9]

([0.16,
  0.07,
  0.21,
  0.05,
  0.0,
  0.0,
  0.3,
  0.14,
  0.0,
  0.05,
  0.0,
  0.02,
  0.0,
  0.0,
  0.0,
  0.0],
 2)

In [6]:
# 填充序列並轉換為張量
input_data = torch.tensor([feature for feature, _ in encoded_data], dtype=torch.float32)
target_personality = torch.tensor([target for _, target in encoded_data], dtype=torch.int64)  # 使用int64类型，因为它是类别标签

In [7]:
# 定义模型（修改 input_size）
class PersonalityPredictionANN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_prob=0.5):
        super(PersonalityPredictionANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

In [8]:
# 資料集切分為訓練集和驗證集
train_dialogues, val_dialogues, train_target, val_target = train_test_split(input_data, target_personality, test_size=0.15, random_state=42)

# 初始化模型
hidden_size = 128
input_size = len(personality_mapping)+1
output_size = len(personality_mapping)
model = PersonalityPredictionANN(input_size, hidden_size, hidden_size, output_size)

# 定义損失函數和優化器 (CEL:分類問題 MSE:回归问题)
criterion = nn.CrossEntropyLoss()
weight_decay = 0.01
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=weight_decay)

In [9]:
#早停
patience = 10  # 設定早期停止的耐心值
best_val_loss = float('inf')
counter = 0  # 用於計算連續的驗證損失沒有改善的次數

In [10]:
!pip install pyelm

In [11]:
def train_elm(X_train, y_train, n_hidden):
    # 随机初始化输入层到隐藏层的权重矩阵
    input_weights = np.random.rand(X_train.shape[1], n_hidden)

    # 计算隐藏层的输出
    hidden_outputs = np.dot(X_train, input_weights)

    # 计算隐藏层到输出层的权重矩阵
    output_weights = np.linalg.pinv(hidden_outputs).dot(y_train)

    return input_weights, output_weights

In [12]:
def predict_elm(X_test, input_weights, output_weights):
    hidden_outputs = np.dot(X_test, input_weights)
    y_pred = np.dot(hidden_outputs, output_weights)
    return y_pred

In [13]:
input_weights, output_weights = train_elm(train_dialogues, train_target, 100)
y_pred_tra = predict_elm(train_dialogues, input_weights, output_weights)
y_pred_val = predict_elm(val_dialogues, input_weights, output_weights)

In [14]:
print(y_pred_val)

[4.86954168 4.82586411 3.17827376 ... 6.44912336 4.56641184 6.04857192]


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred_train_rounded = [round(pred) for pred in y_pred_tra]

# 计算准确度
accuracy = accuracy_score(train_target, y_pred_train_rounded)
print("Accuracy:", accuracy)

# 计算精确度
precision = precision_score(train_target, y_pred_train_rounded, average='macro')
print("Precision:", precision)

# 计算召回率
recall = recall_score(train_target, y_pred_train_rounded, average='macro')
print("Recall:", recall)

# 计算F1分数
f1 = f1_score(train_target, y_pred_train_rounded, average='macro')
print("F1 Score:", f1)

Accuracy: 0.11082474226804123
Precision: 0.0848617701493841
Recall: 0.08161942448159668
F1 Score: 0.04979349890753973


C:\Users\JenMing\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\JenMing\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# 将 elm_predictions 添加为特征

svm_predictions_train = torch.tensor(y_pred_tra, dtype=input_data.dtype, device=input_data.device)

svm_predictions_train = svm_predictions_train.view(-1, 1)

new_train_dialogues = torch.cat((train_dialogues, svm_predictions_train), dim=1)



svm_predictions_val = torch.tensor(y_pred_val, dtype=input_data.dtype, device=input_data.device)

svm_predictions_val = svm_predictions_val.view(-1, 1)

new_val_dialogues = torch.cat((val_dialogues, svm_predictions_val), dim=1)

In [17]:
with open("C:/Users/JenMing/Desktop/MBTI/ANN/Model/pr/ELM/note.txt", "w") as f:
    num_epochs = 60
    dimension_counts = {'E/I': 0,
                        'S/N': 0,
                        'T/F': 0,
                        'J/P': 0}
    item_count = 0
    for epoch in range(num_epochs):
        model.train()  # 將模型設置為訓練模式
        total_loss = 0.0

        for dialogue_batch, target_batch in zip(new_train_dialogues, train_target):
            optimizer.zero_grad()

            outputs = model(dialogue_batch)
            loss = criterion(outputs, target_batch.long())  # 使用交叉熵損失
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        average_loss = total_loss / len(new_train_dialogues)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")
        f.write(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}\n")
        
        # 驗證模型
        model.eval()  # 將模型設置為評估模式
        correct_predictions = 0
        total_samples = len(new_val_dialogues)
        val_loss = 0.0

        with torch.no_grad():
            for dialogue_batch, target_batch in zip(new_val_dialogues, val_target):
                outputs = model(dialogue_batch)
                loss = criterion(outputs, target_batch.long())  # 使用交叉熵損失
                val_loss += loss.item()

                predicted_class = torch.argmax(outputs).item()
                true_class = target_batch.item()

                for personality, value in personality_mapping.items():
                    if value == predicted_class:
                        mbti_labels_pre = personality
                        break
                for personality, value in personality_mapping.items():
                    if value == int(true_class):
                        mbti_labels_tru = personality 
                        break

                for n in range(4):
                    if n == 0:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['E/I'] += 1
                    elif n == 1:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['S/N'] += 1
                    elif n == 2:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['T/F'] += 1
                    elif n == 3:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['J/P'] += 1

                if predicted_class == true_class:
                    correct_predictions += 1

        item_count += total_samples
        average_val_loss = val_loss / len(val_dialogues)
        accuracy = correct_predictions / total_samples
        print(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy*100:.4f}%")
        f.write(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy*100:.4f}%\n")
        
        # 檢查驗證損失是否改善
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            counter = 0
        else:
            counter += 1

        # 如果連續一定次數（耐心值）驗證損失沒有改善，則停止訓練
        if counter >= patience:
            print(f"Early Stopping: Validation loss has not improved for {patience} epochs. Stopping training.")
            break

    EI_counts = dimension_counts['E/I']
    SN_counts = dimension_counts['S/N']
    TF_counts = dimension_counts['T/F']
    JP_counts = dimension_counts['J/P']
    print(f'E.I: {EI_counts}/{item_count} ')
    print('Accuracy: '+ str(EI_counts/item_count)+'\n')
    print(f'S.N: {SN_counts}/{item_count} ')
    print('Accuracy: '+ str(SN_counts/item_count)+'\n')
    print(f'T.F: {TF_counts}/{item_count} ')
    print('Accuracy: '+ str(TF_counts/item_count)+'\n')
    print(f'J.P: {JP_counts}/{item_count} ')
    print('Accuracy: '+ str(JP_counts/item_count)+'\n')
    
    f.write(f'E.I: {EI_counts}/{item_count} ')
    f.write('Accuracy: '+ str(EI_counts/item_count)+'\n')
    f.write(f'S.N: {SN_counts}/{item_count} ')
    f.write('Accuracy: '+ str(SN_counts/item_count)+'\n')
    f.write(f'T.F: {TF_counts}/{item_count} ')
    f.write('Accuracy: '+ str(TF_counts/item_count)+'\n')
    f.write(f'J.P: {JP_counts}/{item_count} ')
    f.write('Accuracy: '+ str(JP_counts/item_count)+'\n')

Epoch [1/60], Loss: 2.2822
Validation Loss: 2.1296, Validation Accuracy: 29.1091%
Epoch [2/60], Loss: 2.1654
Validation Loss: 2.0287, Validation Accuracy: 31.4132%
Epoch [3/60], Loss: 2.0678
Validation Loss: 1.9756, Validation Accuracy: 32.2581%
Epoch [4/60], Loss: 2.0461
Validation Loss: 1.9667, Validation Accuracy: 33.0261%
Epoch [5/60], Loss: 2.0389
Validation Loss: 1.9577, Validation Accuracy: 33.1797%
Epoch [6/60], Loss: 2.0338
Validation Loss: 1.9573, Validation Accuracy: 33.3333%
Epoch [7/60], Loss: 2.0264
Validation Loss: 1.9594, Validation Accuracy: 33.3333%
Epoch [8/60], Loss: 2.0254
Validation Loss: 1.9491, Validation Accuracy: 33.4869%
Epoch [9/60], Loss: 2.0279
Validation Loss: 1.9521, Validation Accuracy: 33.7174%
Epoch [10/60], Loss: 2.0247
Validation Loss: 1.9484, Validation Accuracy: 33.6406%
Epoch [11/60], Loss: 2.0221
Validation Loss: 1.9500, Validation Accuracy: 33.5637%
Epoch [12/60], Loss: 2.0260
Validation Loss: 1.9544, Validation Accuracy: 33.7174%
Epoch [13/60]

In [18]:
torch.save(model.state_dict(), "C:/Users/JenMing/Desktop/MBTI/ANN/Model/pr/ELM/best_model.pth")